In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd drive

/content/drive


In [0]:
cd My\ Drive

/content/drive/My Drive


In [0]:
import numpy as np
import pickle
import pandas as pd

In [0]:
vector_space= np.load('vector_space1.npy')

In [0]:
print(vector_space.shape)

(32743, 2000)


In [0]:
u, sigma, v = np.linalg.svd(vector_space)

In [0]:
k=50

In [0]:
u = u[:,:k]
sigma = np.diag(sigma)
sigma = sigma[:k,:k]
v = v[:k,:]

In [0]:
np.save('u_50',u)
np.save('sigma_50',sigma)
np.save('v_50',v)

In [0]:
np.save('u',u)
np.save('sigma',sigma)
np.save('v',v)

In [0]:
u = np.load('u.npy')
sigma = np.load('sigma.npy')
v = np.load('v.npy')

In [0]:
# rows = vector_space.shape[0]
# cols = vector_space.shape[1]
# print(rows)
# print(cols)
rows = 32743
cols = 2000

In [0]:
file = open('inverse_dict','rb')
inv_dict = pickle.load(file)
file.close()

In [0]:

for i in range(k):
  vect_topic = v[i,:]
  words=[]
  for j in range(rows):
    dot_prod = np.dot(vect_topic,vector_space[j,:])
#     print('dot_prod: '+str(dot_prod))
    words.append((dot_prod,j))
  words.sort(key=lambda x: x[0],reverse=True)
  string =''
  for i1 in range(8):
    string = string + ', '+inv_dict[words[i1][1]]
  string = str(i)+'. '+string
  print(string)

0. , s, wa, said, mr, hi, ha, thi, year
1. , wa, hi, netflix, metro, rural, equiti, ambul, privat
2. , netflix, hast, sarando, network, subscrib, stream, mr, mccord
3. , netflix, said, priebu, hast, trump, rugbi, yuliya, karageorg
4. , wa, priebu, trump, parti, republican, clinton, campaign, elect
5. , parasit, mr, vik, diseas, helminth, hookworm, said, whipworm
6. , parasit, diseas, vik, wa, helminth, hookworm, wada, whipworm
7. , mr, nationstar, mortgag, hi, loan, foreclosur, rugbi, school
8. , s, nationstar, mortgag, perez, polic, mateen, radiat, foreclosur
9. , nationstar, mortgag, priebu, foreclosur, calib, loan, foreclos, auction
10. , disney, shanghai, iger, said, china, chines, wa, park
11. , britain, european, union, europ, mr, rugbi, british, referendum
12. , disney, priebu, mr, shanghai, radiat, iger, air, plutonium
13. , ozick, mr, yuliya, said, hi, ballet, perez, vitali
14. , rugbi, radiat, air, plutonium, cohn, schoning, contamin, cancer
15. , mr, abort, hi, justic, court

In [0]:
df = pd.read_pickle('df_text.pkl')

In [0]:
df = df.head(2000)

In [0]:
documents = df['Content']

In [0]:
# unique_words=[]
# cnt=0
# for x in all_words:
#     if cnt%10000==0:
#         print(cnt)
#     cnt+=1
#     if x not in unique_words:
#         unique_words.append(x)
unique_words = np.load('unique_words1.npy')

In [0]:
w2id = {}
id2w={}
for i in range(M):
    w2id[unique_words[i]]=i
for x,y in w2id.items():
    id2w[y]=x

In [0]:
docs=[]
currentdoc=[]

In [0]:
for document in documents:
  words_list = document.split(' ')
  currentdoc=[]
  for word in words_list:
    currentdoc.append(w2id[word])
  docs.append(currentdoc)

In [0]:
alpha=5
beta=0.1
N = 2000
M = 32743
K=50
ndz = np.zeros([N,K])+alpha
nzw = np.zeros([K,M])+beta
nz = np.zeros([K])+M*beta

In [0]:
Z=[]

In [0]:
cnt=0
for d,doc in enumerate(docs):
  if cnt%100==0:
    print('cnt: '+str(cnt))
  cnt+=1
  zcurrentDoc = []
  for w in doc:
    pz = np.divide(np.multiply(ndz[d,:],nzw[:,w]),nz)
    z = np.random.multinomial(1,pz/pz.sum()).argmax()
    zcurrentDoc.append(z)
    ndz[d,z]+=1
    nzw[z,w]+=1
    nz[z]+=1
  Z.append(zcurrentDoc)

cnt: 0
cnt: 100
cnt: 200
cnt: 300
cnt: 400
cnt: 500
cnt: 600
cnt: 700
cnt: 800
cnt: 900
cnt: 1000
cnt: 1100
cnt: 1200
cnt: 1300
cnt: 1400
cnt: 1500
cnt: 1600
cnt: 1700
cnt: 1800
cnt: 1900


In [0]:
def gibbsSampling():
  t=0
  for d,doc in enumerate(docs):
    for index,w in enumerate(doc):
      z = Z[d][index]
      ndz[d,z]-=1
      nzw[z,w]-=1
      nz[z]-=1
      pz = np.divide(np.multiply(ndz[d,:],nzw[:,w]),nz)
      z = np.random.multinomial(1,pz/pz.sum()).argmax()
      Z[d][index]=z
      ndz[d,z]+=1
      nzw[z,w]+=1
      nz[z]+=1
    if t%1000==0:
      print('t: '+str(t))
    t+=1

In [0]:
for i in range(10):#26
  gibbsSampling()
  print('i: '+str(i))

t: 0
t: 1000
i: 0
t: 0
t: 1000
i: 1
t: 0
t: 1000
i: 2
t: 0
t: 1000
i: 3
t: 0
t: 1000
i: 4
t: 0
t: 1000
i: 5
t: 0
t: 1000
i: 6
t: 0
t: 1000
i: 7
t: 0
t: 1000
i: 8
t: 0
t: 1000
i: 9


In [0]:
p=0
topicwords=[]
for z in range(0,K):
  ids = nzw[z,:].argsort()
  topicword=[]
  for j in ids:
    topicword.insert(0,id2w[j])
  topicwords.append(topicword[0:min(10,len(topicword))])
  if p%5==0:
    print('p: '+str(p))
  p+=1

p: 0
p: 5
p: 10
p: 15
p: 20
p: 25
p: 30
p: 35
p: 40
p: 45


In [0]:
for x in topicwords:
  string=''
  for y in x:
    string =string+', '+y
  print(string)

, last, work, even, point, can, , free, four, de, better
, wa, peopl, women, just, friend, black, love, didn, famili, go
, water, street, restaur, day, food, brooklyn, like, small, bar, red
, jame, collect, fashion, wear, show, cleveland, night, b, year, dress
, health, china, use, servic, plan, care, provid, oper, help, technolog
, state, gay, right, s, white, ha, action, race, issu, american
, offici, s, c, o, june, n, , month, coupl, announc
, ha, time, ani, side, remain, articl, histori, good, publish, like
, ha, use, year, call, good, want, will, mani, drive, one
, school, student, univers, colleg, educ, high, graduat, degre, institut, class
, year, wednesday, said, two, thursday, member, south, time, monday, appear
, also, two, home, place, come, commun, northern, mayor, strong, ireland
, said, citi, resid, test, sever, accord, hospit, custom, outsid, north
, olymp, year, sport, russian, time, track, world, friday, state, f
, offic, polic, wa, investig, charg, case, justic, matee

In [0]:
#plsa
K=35
N = 2000
M = 32743
from collections import Counter 
X = np.zeros([N,M])
for j in range(N):
    x = documents[j].split(' ')
    c = Counter(x)
    for i in range(M):
        X[j][i]=c[unique_words[i]]
    if j%100==0:
        print(j)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900


In [0]:
lamda = np.random.rand(N,K)
for i in range(N):
  normalization = np.sum(lamda[i,:])
  for j in range(0,K):
    lamda[i,j] = lamda[i,j]/normalization
theta = np.random.rand(K,M)
for i in range(0,K):
  normalization = sum(theta[i,:])
  for j in range(0,M):
    theta[i,j] = theta[i,j]/normalization

In [0]:
p=np.zeros((N,M,K))

In [0]:
dict_list=[]

In [0]:
for i in range(N):
  doc_word={}
  for j in range(M):
    if X[i][j]>0:
      doc_word[j]=np.zeros(K)
  dict_list.append(doc_word)

In [0]:
word_doc = {}
for j in range(M):
  word_doc[j]=[]
  for i in range(N):
    if X[i][j]>0:
      word_doc[j].append(i)

In [0]:
def EStep():
  for i in range(N):
    for j in dict_list[i]:
      den=0
      for k in range(K):
        dict_list[i][j][k]=theta[k,j]*lamda[i,k]
        den += dict_list[i][j][k]
      if den==0:
        for k in range(K):
          dict_list[i][j][k]=0
      else:
        for k in range(K):
          dict_list[i][j][k]=dict_list[i][j][k]/den
    if i%1000==0:
      print('e_i: '+str(i))

In [0]:
def MStep():
  for k in range(K):
    den=0
    for j in range(M):
      theta[k,j] =0;
      for i in word_doc[j]:
        if X[i][j]>0:
          theta[k,j] += X[i,j]*dict_list[i][j][k]
        den += theta[k,j]
    if den==0:
      for j in range(M):
        theta[k,j] = 1.0/M
    else:
      for j in range(M):
        theta[k,j] = theta[k,j]/den
  for i in range(N):
    for k in range(K):
      lamda[i,k]=0
      den=0
      for j in dict_list[i]:
        lamda[i,k] += X[i,j]*dict_list[i][j][k]
        den += X[i,j]
      if den==0:
        lamda[i,k] = 1.0/K
      else:
        lamda[i,k] /= den
    if i%100==0:
      print('m_i: '+str(i))

In [0]:
for i in range(10):#20
  EStep()
  MStep()
  print('i: '+str(i))

e_i: 0
e_i: 1000
m_i: 0
m_i: 100
m_i: 200
m_i: 300
m_i: 400
m_i: 500
m_i: 600
m_i: 700
m_i: 800
m_i: 900
m_i: 1000
m_i: 1100
m_i: 1200
m_i: 1300
m_i: 1400
m_i: 1500
m_i: 1600
m_i: 1700
m_i: 1800
m_i: 1900
i: 0
e_i: 0
e_i: 1000
m_i: 0
m_i: 100
m_i: 200
m_i: 300
m_i: 400
m_i: 500
m_i: 600
m_i: 700
m_i: 800
m_i: 900
m_i: 1000
m_i: 1100
m_i: 1200
m_i: 1300
m_i: 1400
m_i: 1500
m_i: 1600
m_i: 1700
m_i: 1800
m_i: 1900
i: 1
e_i: 0
e_i: 1000
m_i: 0
m_i: 100
m_i: 200
m_i: 300
m_i: 400
m_i: 500
m_i: 600
m_i: 700
m_i: 800
m_i: 900
m_i: 1000
m_i: 1100
m_i: 1200
m_i: 1300
m_i: 1400
m_i: 1500
m_i: 1600
m_i: 1700
m_i: 1800
m_i: 1900
i: 2
e_i: 0
e_i: 1000
m_i: 0
m_i: 100
m_i: 200
m_i: 300
m_i: 400
m_i: 500
m_i: 600
m_i: 700
m_i: 800
m_i: 900
m_i: 1000
m_i: 1100
m_i: 1200
m_i: 1300
m_i: 1400
m_i: 1500
m_i: 1600
m_i: 1700
m_i: 1800
m_i: 1900
i: 3
e_i: 0
e_i: 1000
m_i: 0
m_i: 100
m_i: 200
m_i: 300
m_i: 400
m_i: 500
m_i: 600
m_i: 700
m_i: 800
m_i: 900
m_i: 1000
m_i: 1100
m_i: 1200
m_i: 1300
m_i: 1400
m_i: 

In [0]:
K

35

In [0]:
topicwords = []
for i in range(K):
  ids = theta[i,:].argsort()
  topicword = []
  for j in ids:
    topicword.insert(0,id2w[j])
  topicwords.append(topicword[0:min(10,len(topicword))])

In [0]:
for x in topicwords:
  string=''
  for y in x:
    string =string+', '+y
  print(string)

, mr, trump, hi, s, clinton, campaign, republican, ha, parti, donald
, s, said, wa, new, custom, compani, can, ha, will, also
, percent, s, market, econom, voter, poll, wa, vote, economi, trade
, s, said, like, food, ha, street, can, one, thi, restaur
, wa, s, hi, ms, thi, like, new, mr, book, year
, european, s, britain, union, said, europ, ha, countri, british, leav
, game, hi, s, wa, season, team, said, run, play, leagu
, compani, s, bank, ha, year, will, said, percent, new, thi
, mr, s, said, wa, year, ha, thi, social, ms, new
, s, said, insur, car, wa, unit, driver, compani, will, vehicl
, court, s, justic, law, case, said, state, wa, decis, suprem
, t, s, like, thi, ha, will, can, one, time, make
, said, hi, wa, s, year, thi, ha, time, dr, one
, wa, said, s, hi, met, go, t, rey, just, can
, mr, s, said, china, wa, ha, hi, chines, hong, work
, s, said, one, year, ha, will, peopl, project, wa, can
, s, wa, art, artist, thi, t, work, hi, show, one
, said, mr, hi, wa, polic, s, offic